## Data Cleaning and Wrangling

In this notebook, we will examine the dataset for any empty fields and missing data, then drop unnecessary columns and fix the missing fields 

In [11]:
#Import necessary packages and display parameters

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import us
from datetime import datetime
from pytz import timezone
import pytz
import time
import operator

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', 200)


In [12]:
# Load the dataset and remove the first column

df_merged = pd.read_csv('merged_trim_dataset_Apr2017_Jun2017.csv', index_col=False)
df_merged = df_merged.drop('Unnamed: 0', axis=1)

/Users/cuthlo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (74,75,77,79,81) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [13]:
#Shows the shape of dataset and first 5 rows of data for inspection

df_merged.shape

(946418, 93)

In [14]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,OriginTimeZone,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
0,2017,2,4,1,6,2017-04-01,EV,20366,EV,N936EV,5555,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,11617,1161706,31617,EWN,"New Bern/Morehead/Beaufort, NC",NC,37,North Carolina,36,1953,1952.0,-1.0,0.0,0.0,-1.0,1900-1959,17.0,2009.0,2112.0,5.0,2126,2117.0,-9.0,0.0,0.0,-1.0,2100-2159,0.0,NaN,0.0,93.0,85.0,63.0,1.0,433.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,861.0,2017-04-01 23,ATL,2017-04-01 23:52,-84.4418,33.6301,71.1,46,40.65,310.00,8.00,0.00,30.05,1017.10,10,M,FEW,SCT,M,M,20000.00,25000.00,M,M,M,KATL 012352Z 31008KT 10SM FEW200 SCT250 22/08 ...,2017-04-01 23
1,2017,2,4,1,6,2017-04-01,B6,20409,B6,N586JB,927,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,1505,1511.0,6.0,6.0,0.0,0.0,1500-1559,24.0,1535.0,1748.0,13.0,1751,1801.0,10.0,10.0,0.0,0.0,1700-1759,0.0,NaN,0.0,166.0,170.0,133.0,1.0,937.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,271.0,2017-04-01 19,EWR,2017-04-01 19:51,-74.1693,40.6827,48.9,35.1,58.75,350.00,16.00,0.00,29.87,1011.30,10,20.00,SCT,BKN,M,M,3800.00,5000.00,M,M,M,KEWR 011951Z 35016G20KT 10SM SCT038 BKN050 09/...,2017-04-01 19
2,2017,2,4,1,6,2017-04-01,B6,20409,B6,N534JB,925,12478,1247803,31703,JFK,"New York, NY",NY,36,New York,22,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,1455,1515.0,20.0,20.0,1.0,1.0,1400-1459,32.0,1547.0,1806.0,3.0,1804,1809.0,5.0,5.0,0.0,0.0,1800-1859,0.0,NaN,0.0,189.0,174.0,139.0,1.0,1005.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,241.0,2017-04-01 18,JFK,2017-04-01 18:51,-73.7622,40.6386,45.0,35.1,68.05,340.00,18.00,0.00,29.85,1010.90,10,M,OVC,M,M,M,2400.00,M,M,M,M,KJFK 011851Z 34018KT 10SM OVC024 07/02 A2985 R...,2017-04-01 18
3,2017,2,4,1,6,2017-04-01,B6,20409,B6,N537JB,921,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,14027,1402702,34027,PBI,"West Palm Beach/Palm Beach, FL",FL,12,Florida,33,1835,NaN,NaN,NaN,NaN,NaN,1800-1859,NaN,NaN,NaN,NaN,2152,NaN,NaN,NaN,NaN,NaN,2100-2159,1.0,B,0.0,197.0,NaN,NaN,1.0,1197.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,268.0,2017-04-01 22,BOS,2017-04-01 22:24,-71.0097,42.3606,34.0,33.1,96.46,360.00,14.00,0.00,29.94,M,2.5,M,BKN,OVC,M,M,1000.00,1400.00,M,M,-SN BR,KBOS 012224Z 36014KT 2 1/2SM -SN BR BKN010 OVC...,2017-04-01 22
4,2017,2,4,1,6,2017-04-01,B6,20409,B6,N537JB,921,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,14027,1402702,34027,PBI,"West Palm Beach/Palm Beach, FL",FL,12,Florida,33,1835,NaN,NaN,NaN,NaN,NaN,1800-1859,NaN,NaN,NaN,NaN,2152,NaN,NaN,NaN,NaN,NaN,2100-2159,1.0,B,0.0,197.0,NaN,NaN,1.0,1197.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,268.0,2017-04-01 22,BOS,2017-04-01 22:54,-71.0097,42.3606,33.1,33.1,100,350.00,10.00,0.01,29.95,1014.30,2.5,M,BKN,OVC,M,M,1000.00,1500.00,M,M,-SN BR,KBOS 012254Z 35010KT 2 1/2SM -SN BR BKN010 OVC...,2017-04-01 22


### Remove duplicated rows

In [15]:
#Remove duplication rows after merging datasets

df_merged = df_merged.drop_duplicates(subset=['FlightDate', 'TailNum','FlightNum','Origin','Dest'])

In [16]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 803795 entries, 0 to 946417
Data columns (total 93 columns):
Year                    803795 non-null int64
Quarter                 803795 non-null int64
Month                   803795 non-null int64
DayofMonth              803795 non-null int64
DayOfWeek               803795 non-null int64
FlightDate              803795 non-null object
UniqueCarrier           803795 non-null object
AirlineID               803795 non-null int64
Carrier                 803795 non-null object
TailNum                 803514 non-null object
FlightNum               803795 non-null int64
OriginAirportID         803795 non-null int64
OriginAirportSeqID      803795 non-null int64
OriginCityMarketID      803795 non-null int64
Origin                  803795 non-null object
OriginCityName          803795 non-null object
OriginState             803795 non-null object
OriginStateFips         803795 non-null int64
OriginStateName         803795 non-null object
OriginW

In [17]:
#Check percentage of empty fields in each column

frac_nulls = {col: (sum(df_merged[col].isnull()/len(df_merged))) for col in df_merged.columns}
frac_nulls = sorted(frac_nulls.items(), key = operator.itemgetter(1))
frac_nulls

[('Year', 0.0),
 ('Quarter', 0.0),
 ('Month', 0.0),
 ('DayofMonth', 0.0),
 ('DayOfWeek', 0.0),
 ('FlightDate', 0.0),
 ('UniqueCarrier', 0.0),
 ('AirlineID', 0.0),
 ('Carrier', 0.0),
 ('FlightNum', 0.0),
 ('OriginAirportID', 0.0),
 ('OriginAirportSeqID', 0.0),
 ('OriginCityMarketID', 0.0),
 ('Origin', 0.0),
 ('OriginCityName', 0.0),
 ('OriginState', 0.0),
 ('OriginStateFips', 0.0),
 ('OriginStateName', 0.0),
 ('OriginWac', 0.0),
 ('DestAirportID', 0.0),
 ('DestAirportSeqID', 0.0),
 ('DestCityMarketID', 0.0),
 ('Dest', 0.0),
 ('DestCityName', 0.0),
 ('DestState', 0.0),
 ('DestStateFips', 0.0),
 ('DestStateName', 0.0),
 ('DestWac', 0.0),
 ('CRSDepTime', 0.0),
 ('DepTimeBlk', 0.0),
 ('CRSArrTime', 0.0),
 ('ArrTimeBlk', 0.0),
 ('Cancelled', 0.0),
 ('Diverted', 0.0),
 ('CRSElapsedTime', 0.0),
 ('Flights', 0.0),
 ('Distance', 0.0),
 ('DistanceGroup', 0.0),
 ('IsDelay', 0.0),
 ('OriginTimeZone', 0.0),
 ('AptPreviousDateDelay', 0.0),
 ('AptNoOfFlight', 0.0),
 ('UTCFlightDateTime', 0.0),
 ('stat

In [18]:
(df_merged.Cancelled.sum())/len(df_merged)

0.011749264426875012

NaN in DepDelay and other fields are due to flight cancelled, therefore removing all cancelled flights will removed most of the NaN in those columns.

NaN in ArrDelay and other related fieds are due to flight diverted, and we are interested only in departure delay. Diverted flight will still be included. These columns will be dropped due to unrelated to our prediction.

In [19]:
#Removed flights which was cancelled

df_merged = df_merged[~(df_merged.Cancelled == 1)]


In [20]:
%%time

df_merged = df_merged[~pd.isnull(df_merged.TailNum)]

CPU times: user 393 ms, sys: 240 ms, total: 633 ms
Wall time: 670 ms


In [21]:
df_merged.shape

(794351, 93)

In [22]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,OriginTimeZone,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
0,2017,2,4,1,6,2017-04-01,EV,20366,EV,N936EV,5555,10397,1039705,30397,ATL,"Atlanta, GA",GA,13,Georgia,34,11617,1161706,31617,EWN,"New Bern/Morehead/Beaufort, NC",NC,37,North Carolina,36,1953,1952.0,-1.0,0.0,0.0,-1.0,1900-1959,17.0,2009.0,2112.0,5.0,2126,2117.0,-9.0,0.0,0.0,-1.0,2100-2159,0.0,NaN,0.0,93.0,85.0,63.0,1.0,433.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,861.0,2017-04-01 23,ATL,2017-04-01 23:52,-84.4418,33.6301,71.1,46,40.65,310.00,8.00,0.0,30.05,1017.10,10,M,FEW,SCT,M,M,20000.00,25000.00,M,M,M,KATL 012352Z 31008KT 10SM FEW200 SCT250 22/08 ...,2017-04-01 23
1,2017,2,4,1,6,2017-04-01,B6,20409,B6,N586JB,927,11618,1161802,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,1505,1511.0,6.0,6.0,0.0,0.0,1500-1559,24.0,1535.0,1748.0,13.0,1751,1801.0,10.0,10.0,0.0,0.0,1700-1759,0.0,NaN,0.0,166.0,170.0,133.0,1.0,937.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,271.0,2017-04-01 19,EWR,2017-04-01 19:51,-74.1693,40.6827,48.9,35.1,58.75,350.00,16.00,0.0,29.87,1011.30,10,20.00,SCT,BKN,M,M,3800.00,5000.00,M,M,M,KEWR 011951Z 35016G20KT 10SM SCT038 BKN050 09/...,2017-04-01 19
2,2017,2,4,1,6,2017-04-01,B6,20409,B6,N534JB,925,12478,1247803,31703,JFK,"New York, NY",NY,36,New York,22,15304,1530402,33195,TPA,"Tampa, FL",FL,12,Florida,33,1455,1515.0,20.0,20.0,1.0,1.0,1400-1459,32.0,1547.0,1806.0,3.0,1804,1809.0,5.0,5.0,0.0,0.0,1800-1859,0.0,NaN,0.0,189.0,174.0,139.0,1.0,1005.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,241.0,2017-04-01 18,JFK,2017-04-01 18:51,-73.7622,40.6386,45.0,35.1,68.05,340.00,18.00,0.0,29.85,1010.90,10,M,OVC,M,M,M,2400.00,M,M,M,M,KJFK 011851Z 34018KT 10SM OVC024 07/02 A2985 R...,2017-04-01 18
5,2017,2,4,1,6,2017-04-01,B6,20409,B6,N368JB,918,12478,1247803,31703,JFK,"New York, NY",NY,36,New York,22,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,1805,2101.0,176.0,176.0,1.0,11.0,1800-1859,21.0,2122.0,2157.0,6.0,1928,2203.0,155.0,155.0,1.0,10.0,1900-1959,0.0,NaN,0.0,83.0,62.0,35.0,1.0,187.0,1,19.0,0.0,0.0,0.0,136.0,NaN,NaN,1,America/New_York,0.0,241.0,2017-04-01 22,JFK,2017-04-01 22:51,-73.7622,40.6386,45.0,37.9,76.01,20.00,9.00,0.0,29.96,1014.40,10,M,BKN,BKN,BKN,BKN,4500.00,5000.00,7000.00,25000.00,M,KJFK 012251Z 02009KT 10SM BKN045 BKN050 BKN070...,2017-04-01 22
10,2017,2,4,1,6,2017-04-01,B6,20409,B6,N976JB,916,14771,1477102,32457,SFO,"San Francisco, CA",CA,6,California,91,12478,1247803,31703,JFK,"New York, NY",NY,36,New York,22,1600,1917.0,197.0,197.0,1.0,12.0,1600-1659,18.0,1935.0,323.0,7.0,35,330.0,175.0,175.0,1.0,11.0,0001-0559,0.0,NaN,0.0,335.0,313.0,288.0,1.0,2586.0,11,4.0,0.0,0.0,0.0,171.0,NaN,NaN,1,America/Los_Angeles,0.0,383.0,2017-04-01 23,SFO,2017-04-01 23:56,-122.3749,37.6190,69.1,46,43.52,290.00,17.00,0.0,29.98,1015.20,10,M,FEW,M,M,M,20000.00,M,M,M,M,KSFO 012356Z 29017KT 10SM FEW200 21/08 A2998 R...,2017-04-01 23


In [23]:
#Create a new column from combining the Carrier code and flight number to represent the real world flight numbers

df_merged.FlightNum = df_merged.Carrier + df_merged.FlightNum.astype(str)

In [24]:
df_merged.IsDelay.sum()/len(df_merged)

0.22248225280763792

In [25]:
df_merged[(df_merged[' alti'] == 'M')]

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,OriginTimeZone,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
300150,2017,2,4,30,7,2017-04-30,EV,20366,EV,N748EV,EV5266,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,15370,1537002,34653,TUL,"Tulsa, OK",OK,40,Oklahoma,73,2022,2019.0,-3.0,0.0,0.0,-1.0,2000-2059,17.0,2036.0,2140.0,4.0,2156,2144.0,-12.0,0.0,0.0,-1.0,2100-2159,0.0,NaN,0.0,154.0,145.0,124.0,1.0,789.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,31.0,342.0,2017-05-01 00,DTW,2017-05-01 00:51,-83.33,42.23,53.6,53.6,100,90.00,5.00,0.0,M,M,3,M,SCT,BKN,OVC,M,300.00,6000.00,8000.00,M,-RA BR,KDTW 010051Z 09005KT 3SM -RA BR SCT003 BKN060 ...,2017-05-01 00
300185,2017,2,4,30,7,2017-04-30,EV,20366,EV,N755EV,EV5219,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,10599,1059904,30599,BHM,"Birmingham, AL",AL,1,Alabama,51,2016,2052.0,36.0,36.0,1.0,2.0,2000-2059,23.0,2115.0,2151.0,5.0,2124,2156.0,32.0,32.0,1.0,2.0,2100-2159,0.0,NaN,0.0,128.0,124.0,96.0,1.0,625.0,3,5.0,0.0,0.0,0.0,27.0,NaN,NaN,1,America/New_York,31.0,342.0,2017-05-01 00,DTW,2017-05-01 00:51,-83.33,42.23,53.6,53.6,100,90.00,5.00,0.0,M,M,3,M,SCT,BKN,OVC,M,300.00,6000.00,8000.00,M,-RA BR,KDTW 010051Z 09005KT 3SM -RA BR SCT003 BKN060 ...,2017-05-01 00
300193,2017,2,4,30,7,2017-04-30,EV,20366,EV,N606LR,EV5209,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,11977,1197703,31977,GRB,"Green Bay, WI",WI,55,Wisconsin,45,2017,2017.0,0.0,0.0,0.0,0.0,2000-2059,29.0,2046.0,2039.0,3.0,2035,2042.0,7.0,7.0,0.0,0.0,2000-2059,0.0,NaN,0.0,78.0,85.0,53.0,1.0,287.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,31.0,342.0,2017-05-01 00,DTW,2017-05-01 00:51,-83.33,42.23,53.6,53.6,100,90.00,5.00,0.0,M,M,3,M,SCT,BKN,OVC,M,300.00,6000.00,8000.00,M,-RA BR,KDTW 010051Z 09005KT 3SM -RA BR SCT003 BKN060 ...,2017-05-01 00
300209,2017,2,4,30,7,2017-04-30,EV,20366,EV,N228PQ,EV5324,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,12992,1299204,32600,LIT,"Little Rock, AR",AR,5,Arkansas,71,2015,2124.0,69.0,69.0,1.0,4.0,2000-2059,12.0,2136.0,2223.0,4.0,2132,2227.0,55.0,55.0,1.0,3.0,2100-2159,0.0,NaN,0.0,137.0,123.0,107.0,1.0,705.0,3,21.0,0.0,0.0,0.0,34.0,NaN,NaN,1,America/New_York,31.0,342.0,2017-05-01 00,DTW,2017-05-01 00:51,-83.33,42.23,53.6,53.6,100,90.00,5.00,0.0,M,M,3,M,SCT,BKN,OVC,M,300.00,6000.00,8000.00,M,-RA BR,KDTW 010051Z 09005KT 3SM -RA BR SCT003 BKN060 ...,2017-05-01 00
300247,2017,2,4,30,7,2017-04-30,NK,20416,NK,N626NK,NK706,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,2005,2004.0,-1.0,0.0,0.0,-1.0,2000-2059,8.0,2012.0,2114.0,7.0,2207,2121.0,-46.0,0.0,0.0,-2.0,2200-2259,0.0,NaN,0.0,302.0,257.0,242.0,1.0,1979.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,31.0,342.0,2017-05-01 00,DTW,2017-05-01 00:51,-83.33,42.23,53.6,53.6,100,90.00,5.00,0.0,M,M,3,M,SCT,BKN,OVC,M,300.00,6000.00,8000.00,M,-RA BR,KDTW 010051Z 09005KT 3SM -RA BR SCT003 BKN060 ...,2017-05-01 00
300272,2017,2,4,30,7,2017-04-30,EV,20366,EV,N712EV,EV5357,11433,1143302,31295,DTW,"Detroit, MI",MI,26,Michigan,43,14524,1452401,34524,RIC,"Richmond, VA",VA,51,Virginia,38,2025,2034.0,9.0,

In [26]:
#By examining the raw METAR both columns ' drct' and ' sknt' are missing are because those are not reported in the METAR

df_merged[(df_merged[' drct'] == 'M') & (df_merged[' sknt'] == 'M')][' metar']

85        KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
305       KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
946       KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
1411      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
1941      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
1950      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
1957      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
2000      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
2044      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
2057      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
2091      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
2099      KLAS 012256Z 10SM FEW085 FEW250 23/M01 A2992 R...
2138      KPHX 012051Z 10SM SCT120 SCT160 SCT200 21/M04 ...
2174      KPHX 012051Z 10SM SCT120 SCT160 SCT200 21/M04 ...
2183      KPHX 012051Z 10SM SCT120 SCT160 SCT200 21/M04 ...
2438      KPHX 012051Z 10SM SCT120 SCT160 SCT200 21/M04 ...
2443      KPHX 012051Z 10SM SCT120 SCT16

In [27]:
df_merged[df_merged[' metar'].str.contains("00000KT")]

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,OriginTimeZone,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
123,2017,2,4,1,6,2017-04-01,B6,20409,B6,N794JB,B6667,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,14254,1425403,34254,PSE,"Ponce, PR",PR,72,Puerto Rico,3,2340,2333.0,-7.0,0.0,0.0,-1.0,2300-2359,11.0,2344.0,203.0,3.0,227,206.0,-21.0,0.0,0.0,-2.0,0001-0559,0.0,NaN,0.0,167.0,153.0,139.0,1.0,1179.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,442.0,2017-04-02 03,MCO,2017-04-02 03:52,-81.3090,28.4294,71.6,64.4,78.04,0.00,0.00,0.0,30.08,M,10,M,BKN,M,M,M,25000.00,M,M,M,M,KMCO 020352Z 00000KT 10SM BKN250 22/18 A3008,2017-04-02 03
154,2017,2,4,1,6,2017-04-01,B6,20409,B6,N318JB,B61494,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,12197,1219702,31703,HPN,"White Plains, NY",NY,36,New York,22,1751,1841.0,50.0,50.0,1.0,3.0,1700-1759,15.0,1856.0,2105.0,3.0,2030,2108.0,38.0,38.0,1.0,2.0,2000-2059,0.0,NaN,0.0,159.0,147.0,129.0,1.0,972.0,4,38.0,0.0,0.0,0.0,0.0,NaN,NaN,1,America/New_York,0.0,442.0,2017-04-01 21,MCO,2017-04-01 21:53,-81.3090,28.4294,87.8,51.8,29.18,0.00,0.00,0.0,29.98,M,10,M,BKN,M,M,M,25000.00,M,M,M,M,KMCO 012153Z 00000KT 10SM BKN250 31/11 A2998,2017-04-01 21
158,2017,2,4,1,6,2017-04-01,B6,20409,B6,N653JB,B61690,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,12478,1247803,31703,JFK,"New York, NY",NY,36,New York,22,1711,1711.0,0.0,0.0,0.0,0.0,1700-1759,15.0,1726.0,1939.0,5.0,2000,1944.0,-16.0,0.0,0.0,-2.0,2000-2059,0.0,NaN,0.0,169.0,153.0,133.0,1.0,944.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,442.0,2017-04-01 21,MCO,2017-04-01 21:53,-81.3090,28.4294,87.8,51.8,29.18,0.00,0.00,0.0,29.98,M,10,M,BKN,M,M,M,25000.00,M,M,M,M,KMCO 012153Z 00000KT 10SM BKN250 31/11 A2998,2017-04-01 21
280,2017,2,4,1,6,2017-04-01,B6,20409,B6,N636JB,B6179,12889,1288903,32211,LAS,"Las Vegas, NV",NV,32,Nevada,85,12954,1295403,32575,LGB,"Long Beach, CA",CA,6,California,91,2025,2019.0,-6.0,0.0,0.0,-1.0,2000-2059,12.0,2031.0,2111.0,4.0,2130,2115.0,-15.0,0.0,0.0,-1.0,2100-2159,0.0,NaN,0.0,65.0,56.0,40.0,1.0,231.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/Los_Angeles,0.0,376.0,2017-04-02 03,LAS,2017-04-02 03:56,-115.1522,36.0801,70.0,33.1,25.53,0.00,0.00,0.0,29.96,1013.00,10,M,CLR,M,M,M,M,M,M,M,M,KLAS 020356Z 00000KT 10SM CLR 21/01 A2996 RMK ...,2017-04-02 03
287,2017,2,4,1,6,2017-04-01,B6,20409,B6,N354JB,B6152,13204,1320402,31454,MCO,"Orlando, FL",FL,12,Florida,33,10721,1072102,30721,BOS,"Boston, MA",MA,25,Massachusetts,13,1700,1658.0,-2.0,0.0,0.0,-1.0,1700-1759,16.0,1714.0,1940.0,9.0,1956,1949.0,-7.0,0.0,0.0,-1.0,1900-1959,0.0,NaN,0.0,176.0,171.0,146.0,1.0,1121.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/New_York,0.0,442.0,2017-04-01 21,MCO,2017-04-01 21:53,-81.3090,28.4294,87.8,51.8,29.18,0.00,0.00,0.0,29.98,M,10,M,BKN,M,M,M,25000.00,M,M,M,M,KMCO 012153Z 00000KT 10SM BKN250 31/11 A2998,2017-04-01 21
299,2017,2,4,1,6,2017-04-01,VX,21171,VX,N283VA,VX925,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,14771,1477102,32457,SFO,"San Francisco, CA",CA,6,California,91,700,653.0,-7.0,0.0,0.0,-1.0,0700-0759,15.0,708.0,806.0,9.0,820,815.0,-5.0,0.0,0.0,-1.0,0800-0859,0

If wind is calm, the code will be 00000KT instead of just missing.

In [28]:
#Remove rows that both wind direction and speed are missing.

df_merged = df_merged[~((df_merged[' drct'] == 'M') & (df_merged[' sknt'] == 'M'))]

#Remove rows that dew point is missing due to instrument error
df_merged = df_merged[~((df_merged[' dwpf'] == 'M'))]

#Remove rows that visibility is missing due to instrument error
df_merged = df_merged[~((df_merged[' vsby'] == 'M'))]

#Remove rows that altimeter setting is missing due to instrument error
df_merged = df_merged[~((df_merged[' alti'] == 'M'))]

#Remove rows of April 1st as there is no delay figure of previous day of March 31st
df_merged = df_merged[~((df_merged['Month'] == 4) & (df_merged['DayofMonth'] == 1))]

In [29]:
#Check no more Missing in both wind direction and speed
df_merged[((df_merged[' drct'] == 'M') & (df_merged[' sknt'] == 'M'))]

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,OriginTimeZone,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr


In [30]:
#Check for rows that ' drct' is having value 'M' that is VRB wind, we will later change it to 999.

df_merged[df_merged[' drct'] == 'M']

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,UniqueCarrier,AirlineID,Carrier,TailNum,FlightNum,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,IsDelay,OriginTimeZone,AptPreviousDateDelay,AptNoOfFlight,UTCFlightDateTime,station,valid,lon,lat,tmpf,dwpf,relh,drct,sknt,p01i,alti,mslp,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx,metar,DateHr
8384,2017,2,4,2,7,2017-04-02,OO,20304,OO,N973SW,OO5243,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,11648,1164802,31648,FCA,"Kalispell, MT",MT,30,Montana,84,1130,1120.0,-10.0,0.0,0.0,-1.0,1100-1159,17.0,1137.0,1332.0,5.0,1348,1337.0,-11.0,0.0,0.0,-1.0,1300-1359,0.0,NaN,0.0,138.0,137.0,115.0,1.0,752.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/Denver,91.0,603.0,2017-04-02 17,DEN,2017-04-02 17:53,-104.6575,39.8328,57.0,28,32.75,M,4.00,0.0,29.87,1009.20,10,M,FEW,FEW,M,M,7000.00,22000.00,M,M,M,KDEN 021753Z VRB04KT 10SM FEW070 FEW220 14/M02...,2017-04-02 17
8386,2017,2,4,2,7,2017-04-02,OO,20304,OO,N925SW,OO5198,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,14679,1467903,33570,SAN,"San Diego, CA",CA,6,California,91,915,902.0,-13.0,0.0,0.0,-1.0,0900-0959,13.0,915.0,943.0,5.0,1019,948.0,-31.0,0.0,0.0,-2.0,1000-1059,0.0,NaN,0.0,64.0,46.0,28.0,1.0,109.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/Los_Angeles,96.0,591.0,2017-04-02 16,LAX,2017-04-02 16:53,-118.3865,33.9382,66.0,54,65.21,M,4.00,0.0,30.02,1016.60,9,M,FEW,FEW,SCT,M,1000.00,23000.00,28000.00,M,M,KLAX 021653Z VRB04KT 9SM FEW010 FEW230 SCT280 ...,2017-04-02 16
8406,2017,2,4,2,7,2017-04-02,OO,20304,OO,N206SY,OO5235,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,11066,1106603,31066,CMH,"Columbus, OH",OH,39,Ohio,44,1005,1120.0,75.0,75.0,1.0,5.0,1000-1059,16.0,1136.0,1556.0,5.0,1459,1601.0,62.0,62.0,1.0,4.0,1400-1459,0.0,NaN,0.0,174.0,161.0,140.0,1.0,1154.0,5,62.0,0.0,0.0,0.0,0.0,NaN,NaN,1,America/Denver,91.0,603.0,2017-04-02 16,DEN,2017-04-02 16:53,-104.6575,39.8328,55.0,30.9,39.63,M,5.00,0.0,29.88,1009.40,10,M,FEW,M,M,M,7000.00,M,M,M,M,KDEN 021653Z VRB05KT 10SM FEW070 13/M01 A2988 ...,2017-04-02 16
8419,2017,2,4,2,7,2017-04-02,OO,20304,OO,N975SW,OO5225,14869,1486903,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,11292,1129202,30325,DEN,"Denver, CO",CO,8,Colorado,82,1010,1006.0,-4.0,0.0,0.0,-1.0,1000-1059,22.0,1028.0,1126.0,8.0,1146,1134.0,-12.0,0.0,0.0,-1.0,1100-1159,0.0,NaN,0.0,96.0,88.0,58.0,1.0,391.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/Denver,27.0,320.0,2017-04-02 16,SLC,2017-04-02 16:54,-111.9700,40.7800,51.1,37.9,60.46,M,4.00,0.0,30.01,1014.20,9,M,FEW,OVC,M,M,3800.00,4700.00,M,M,-RA,KSLC 021654Z VRB04KT 9SM -RA FEW038 OVC047 11/...,2017-04-02 16
8445,2017,2,4,2,7,2017-04-02,OO,20304,OO,N982SW,OO5038,12892,1289205,32575,LAX,"Los Angeles, CA",CA,6,California,91,14689,1468902,34689,SBA,"Santa Barbara, CA",CA,6,California,91,955,947.0,-8.0,0.0,0.0,-1.0,0900-0959,22.0,1009.0,1029.0,5.0,1048,1034.0,-14.0,0.0,0.0,-1.0,1000-1059,0.0,NaN,0.0,53.0,47.0,20.0,1.0,89.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,America/Los_Angeles,96.0,591.0,2017-04-02 16,LAX,2017-04-02 16:53,-118.3865,33.9382,66.0,54,65.21,M,4.00,0.0,30.02,1016.60,9,M,FEW,FEW,SCT,M,1000.00,23000.00,28000.00,M,M,KLAX 021653Z VRB04KT 9SM FEW010 FEW230 SCT280 ...,2017-04-02 16
8477,2017,2,4,2,7,2017-04-02,AA,19805,AA,N925AA,AA521,11057,1105703,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,14107,1410702,30466,PHX,"Phoenix, AZ",AZ,4,Arizona,81,1436,1438.0,2.

The model is for prediction of departure delay, any post flight information are irrelevant, therefore WheelsOn, TaxiIn, ArrTime, ActualElapsedTime, AirTime can be dropped.

In [31]:
%%time

df_merged.drop(['FlightDate','TailNum','FlightNum','Flights',
                'CarrierDelay','WeatherDelay','NASDelay','SecurityDelay','LateAircraftDelay',
                'CancellationCode','FirstDepTime','TotalAddGTime','UniqueCarrier','AirlineID',
                'OriginAirportID','OriginAirportSeqID','OriginCityMarketID','OriginStateFips',
                'OriginStateName','OriginCityName','OriginTimeZone',
                'DestAirportID','DestAirportSeqID','DestCityMarketID','DestStateFips',
                'DestStateName','DestCityName',
                'DepTime','DepDelay','DepDelayMinutes','DepDel15','DepartureDelayGroups',
                'ArrDelay','ArrDelayMinutes','ArrDel15','ArrivalDelayGroups',
                'ArrTime','ActualElapsedTime','AirTime',
                'WheelsOn','TaxiIn','TaxiOut','WheelsOff',
                'Cancelled','Diverted','UTCFlightDateTime',
                'valid','station','lat','lon',' mslp','DateHr',' metar'], 
               axis=1, inplace=True)

CPU times: user 280 ms, sys: 207 ms, total: 487 ms
Wall time: 508 ms


In [32]:
df_merged.shape

(784754, 40)

In [33]:
df_merged.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,Carrier,Origin,OriginState,OriginWac,Dest,DestState,DestWac,CRSDepTime,DepTimeBlk,CRSArrTime,ArrTimeBlk,CRSElapsedTime,Distance,DistanceGroup,IsDelay,AptPreviousDateDelay,AptNoOfFlight,tmpf,dwpf,relh,drct,sknt,p01i,alti,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx
8381,2017,2,4,2,7,OO,ORD,IL,41,MBS,MI,43,1345,1300-1359,1557,1500-1559,72.0,222.0,1,0,85.0,715.0,57.9,34,40.43,140.00,10.00,0.0,30.06,10,M,BKN,BKN,OVC,M,8500.00,16000.00,25000.00,M,M
8382,2017,2,4,2,7,AA,PHX,AZ,81,SEA,WA,93,2245,2200-2259,153,0001-0559,188.0,1107.0,5,0,56.0,478.0,73.9,32,21.4,250.00,4.00,0.0,29.83,10,M,CLR,M,M,M,M,M,M,M,M
8383,2017,2,4,2,7,WN,MSP,MN,63,BWI,MD,35,650,0600-0659,1010,1000-1059,140.0,936.0,4,0,22.0,378.0,45.0,34,65.13,180.00,5.00,0.0,30.05,10,M,BKN,M,M,M,10000.00,M,M,M,M
8384,2017,2,4,2,7,OO,DEN,CO,82,FCA,MT,84,1130,1100-1159,1348,1300-1359,138.0,752.0,4,0,91.0,603.0,57.0,28,32.75,M,4.00,0.0,29.87,10,M,FEW,FEW,M,M,7000.00,22000.00,M,M,M
8385,2017,2,4,2,7,OO,ATL,GA,34,ORD,IL,41,1850,1800-1859,1957,1900-1959,127.0,606.0,3,0,70.0,1051.0,81.0,45,28.15,140.00,6.00,0.0,29.97,10,M,BKN,BKN,M,M,20000.00,25000.00,M,M,M


In [34]:
#Replacing the "M" missing flag with -1 or 0 in gust for wind

df_merged[' drct'] = df_merged[' drct'].replace({'M':-1})
df_merged[' gust'] = df_merged[' gust'].replace({'M':0})
df_merged[' skyl1'] = df_merged[' skyl1'].replace({'M':-1})
df_merged[' skyl2'] = df_merged[' skyl2'].replace({'M':-1})
df_merged[' skyl3'] = df_merged[' skyl3'].replace({'M':-1})
df_merged[' skyl4'] = df_merged[' skyl4'].replace({'M':-1})

#Reformat the CRS departure time and arrival time to capture only the hours instead of the hours minutes format for modeling purpose

df_merged.CRSDepTime = df_merged.CRSDepTime.apply(lambda x: str(int(x)).zfill(4))
df_merged.CRSDepTime = df_merged.CRSDepTime.apply(lambda x: x[:2])
df_merged.CRSDepTime = pd.to_numeric(df_merged.CRSDepTime, errors='coerce')

df_merged.CRSArrTime = df_merged.CRSArrTime.apply(lambda x: str(int(x)).zfill(4))
df_merged.CRSArrTime = df_merged.CRSArrTime.apply(lambda x: x[:2])
df_merged.CRSArrTime = pd.to_numeric(df_merged.CRSArrTime, errors='coerce')



In [35]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 784754 entries, 8381 to 946417
Data columns (total 40 columns):
Year                    784754 non-null int64
Quarter                 784754 non-null int64
Month                   784754 non-null int64
DayofMonth              784754 non-null int64
DayOfWeek               784754 non-null int64
Carrier                 784754 non-null object
Origin                  784754 non-null object
OriginState             784754 non-null object
OriginWac               784754 non-null int64
Dest                    784754 non-null object
DestState               784754 non-null object
DestWac                 784754 non-null int64
CRSDepTime              784754 non-null int64
DepTimeBlk              784754 non-null object
CRSArrTime              784754 non-null int64
ArrTimeBlk              784754 non-null object
CRSElapsedTime          784754 non-null float64
Distance                784754 non-null float64
DistanceGroup           784754 non-null int64
I

In [36]:
df_merged.head(10)


,Year,Quarter,Month,DayofMonth,DayOfWeek,Carrier,Origin,OriginState,OriginWac,Dest,DestState,DestWac,CRSDepTime,DepTimeBlk,CRSArrTime,ArrTimeBlk,CRSElapsedTime,Distance,DistanceGroup,IsDelay,AptPreviousDateDelay,AptNoOfFlight,tmpf,dwpf,relh,drct,sknt,p01i,alti,vsby,gust,skyc1,skyc2,skyc3,skyc4,skyl1,skyl2,skyl3,skyl4,presentwx
8381,2017,2,4,2,7,OO,ORD,IL,41,MBS,MI,43,13,1300-1359,15,1500-1559,72.0,222.0,1,0,85.0,715.0,57.9,34,40.43,140.00,10.00,0.0,30.06,10,0,BKN,BKN,OVC,M,8500.00,16000.00,25000.00,-1,M
8382,2017,2,4,2,7,AA,PHX,AZ,81,SEA,WA,93,22,2200-2259,1,0001-0559,188.0,1107.0,5,0,56.0,478.0,73.9,32,21.4,250.00,4.00,0.0,29.83,10,0,CLR,M,M,M,-1,-1,-1,-1,M
8383,2017,2,4,2,7,WN,MSP,MN,63,BWI,MD,35,6,0600-0659,10,1000-1059,140.0,936.0,4,0,22.0,378.0,45.0,34,65.13,180.00,5.00,0.0,30.05,10,0,BKN,M,M,M,10000.00,-1,-1,-1,M
8384,2017,2,4,2,7,OO,DEN,CO,82,FCA,MT,84,11,1100-1159,13,1300-1359,138.0,752.0,4,0,91.0,603.0,57.0,28,32.75,-1,4.00,0.0,29.87,10,0,FEW,FEW,M,M,7000.00,22000.00,-1,-1,M
8385,2017,2,4,2,7,OO,ATL,GA,34,ORD,IL,41,18,1800-1859,19,1900-1959,127.0,606.0,3,0,70.0,1051.0,81.0,45,28.15,140.00,6.00,0.0,29.97,10,0,BKN,BKN,M,M,20000.00,25000.00,-1,-1,M
8386,2017,2,4,2,7,OO,LAX,CA,91,SAN,CA,91,9,0900-0959,10,1000-1059,64.0,109.0,1,0,96.0,591.0,66.0,54,65.21,-1,4.00,0.0,30.02,9,0,FEW,FEW,SCT,M,1000.00,23000.00,28000.00,-1,M
8387,2017,2,4,2,7,WN,MCO,FL,33,SJU,PR,3,11,1100-1159,13,1300-1359,160.0,1189.0,5,0,70.0,411.0,86.0,66.2,51.72,100.00,9.00,0.0,30.08,10,0,SCT,SCT,BKN,M,5000.00,15000.00,25000.00,-1,M
8388,2017,2,4,2,7,WN,MSP,MN,63,DEN,CO,82,17,1700-1759,18,1800-1859,125.0,680.0,3,0,22.0,378.0,54.0,44.1,69.09,180.00,8.00,0.0,29.81,10,18.00,OVC,M,M,M,2200.00,-1,-1,-1,M
8390,2017,2,4,2,7,OO,LAX,CA,91,SBP,CA,91,14,1400-1459,15,1500-1559,57.0,155.0,1,0,96.0,591.0,63.0,55.9,77.61,260.00,11.00,0.0,29.97,9,0,FEW,FEW,M,M,300.00,20000.00,-1,-1,M
8391,2017,2,4,2,7,OO,DEN,CO,82,RKS,WY,88,18,1800-1859,19,1900-1959,66.0,260.0,2,0,91.0,603.0,55.0,33.1,43.31,50.00,7.00,0.0,29.75,10,0,BKN,BKN,BKN,M,9000.00,14000.00,22000.00,-1,M


In [37]:
#Check for any empty fields again

frac_nulls = {col: (sum(df_merged[col].isnull()/len(df_merged))) for col in df_merged.columns}
frac_nulls = sorted(frac_nulls.items(), key = operator.itemgetter(1))
frac_nulls

[('Year', 0.0),
 ('Quarter', 0.0),
 ('Month', 0.0),
 ('DayofMonth', 0.0),
 ('DayOfWeek', 0.0),
 ('Carrier', 0.0),
 ('Origin', 0.0),
 ('OriginState', 0.0),
 ('OriginWac', 0.0),
 ('Dest', 0.0),
 ('DestState', 0.0),
 ('DestWac', 0.0),
 ('CRSDepTime', 0.0),
 ('DepTimeBlk', 0.0),
 ('CRSArrTime', 0.0),
 ('ArrTimeBlk', 0.0),
 ('CRSElapsedTime', 0.0),
 ('Distance', 0.0),
 ('DistanceGroup', 0.0),
 ('IsDelay', 0.0),
 ('AptPreviousDateDelay', 0.0),
 ('AptNoOfFlight', 0.0),
 ('tmpf', 0.0),
 (' dwpf', 0.0),
 (' relh', 0.0),
 (' drct', 0.0),
 (' sknt', 0.0),
 (' p01i', 0.0),
 (' alti', 0.0),
 (' vsby', 0.0),
 (' gust', 0.0),
 (' skyc1', 0.0),
 (' skyc2', 0.0),
 (' skyc3', 0.0),
 (' skyc4', 0.0),
 (' skyl1', 0.0),
 (' skyl2', 0.0),
 (' skyl3', 0.0),
 (' skyl4', 0.0),
 (' presentwx', 0.0)]

In [38]:
#Move IsDelay column which will be the target column to the back

cols = list(df_merged.columns.values) #Make a list of all of the columns in the df
cols.pop(cols.index('IsDelay')) #Remove IsDelay from list
df_merged = df_merged[cols+['IsDelay']]

In [39]:
%%time

df_merged.to_csv('cleaned_dataset.csv')

CPU times: user 11.4 s, sys: 486 ms, total: 11.9 s
Wall time: 12.2 s


After all the data cleaning and wrangling procedures done, save to a csv file for actual machine learning model training and prediction task.